<div align="center">
  <h2>Міністерство освіти і науки України</h2>
  <h2>Національний технічний університет України</h2>
  <h2>«Київський політехнічний інститут ім. Ігоря Сікорського»</h2>
  <h2>Факультет інформатики та обчислювальної техніки</h2>
  <h2>Кафедра обчислювальної техніки</h2>
  <br>
</div>

<div align="right">
    <br>
    <br>
<center>    
<h2>Лабораторна робота №3</h2>
</center>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
    <br>
Виконав: <br>
Студент ІІ курсу ФІОТ <br>
групи ІО-34 <br>
Рибачок Михайло Володимирович <br>
Номер у списку - 17 <br><br>

<center>
Київ - 2025
</center>

# Робота з пропущеними даними 

## Мета лабораторної роботи: 
#### Навчитися працювати з наборами даних, які містять відсутні або помилкові дані. 

## Практична реалізація: 
#### Відпрацювати практичні підходи при роботі з реальними даними. 

In [1]:
# Завантаження датасету для Data Cleaning з kaggle для виконання лабораторної роботи
import kagglehub

# Download latest version
path = kagglehub.dataset_download("martinkanju/dirty-dataset-to-practice-data-cleaning")

print("Path to dataset files:", path)

100%|██████████| 1.21k/1.21k [00:00<00:00, 1.23MB/s]

Extracting files...
Path to dataset files: C:\Users\Mishanya\.cache\kagglehub\datasets\martinkanju\dirty-dataset-to-practice-data-cleaning\versions\1


In [4]:
# Підкючення потрібних бібліотек
import pandas as pd 
import numpy as np
import re

## Перший етап:
#### Aналізу даних

In [2]:
# Перевірка датасету
data_for_cleaning = pd.read_csv("my_file.csv")
data_for_cleaning.head(9) # відображення з пропусками у даних

,Rank,Peak,All Time Peak,Actual gross,Actual gross(in 2022 dollars),Artist,Tour title,Year(s),Shows,Average gross,Ref.
0,1,1,2,"$780,000,000","$780,000,000",Taylor Swift,The Eras Tour †,2023–2024,56,"$13,928,571",[1]
1,2,1,7[2],"$579,800,000","$579,800,000",Beyoncé,Renaissance World Tour,2023,56,"$10,353,571",[3]
2,3,1[4],2[5],"$411,000,000","$560,622,615",Madonna,Sticky & Sweet Tour ‡[4][a],2008–2009,85,"$4,835,294",[6]
3,4,2[7],10[7],"$397,300,000","$454,751,555",Pink,Beautiful Trauma World Tour,2018–2019,156,"$2,546,795",[7]
4,5,2[4],NaN,"$345,675,146","$402,844,849",Taylor Swift,Reputation Stadium Tour,2018,53,"$6,522,173",[8]
5,6,2[4],10[9],"$305,158,363","$388,978,496",Madonna,The MDNA Tour,2012,88,"$3,467,709",[9]
6,7,2[10],NaN,"$280,000,000","$381,932,682",Celine Dion,Taking Chances World Tour,2008–2009,131,"$2,137,405",[11]
7,7,NaN,NaN,"$257,600,000","$257,600,000",Pink,Summer Carnival †,2023–2024,41,"$6,282,927",[12]
8,9,NaN,NaN,"$256,084,556","$312,258,401",Beyoncé,The Formation World Tour,2016,49,"$5,226,215",[13]


In [42]:
# Найчастіший артист
most_common_artist = data_for_cleaning['Artist'].value_counts().idxmax()
count = data_for_cleaning['Artist'].value_counts().max()
count, most_common_artist

(4, 'Taylor Swift')

In [43]:
data_for_cleaning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Rank                           20 non-null     int64 
 1   Peak                           9 non-null      object
 2   All Time Peak                  6 non-null      object
 3   Actual gross                   20 non-null     object
 4   Actual gross(in 2022 dollars)  20 non-null     object
 5   Artist                         20 non-null     object
 6   Tour title                     20 non-null     object
 7   Year(s)                        20 non-null     object
 8   Shows                          20 non-null     int64 
 9   Average gross                  20 non-null     object
 10  Ref.                           20 non-null     object
dtypes: int64(2), object(9)
memory usage: 1.8+ KB


## Другий етап: 
#### Дозволяє визначити кількість відсутніх даних у кожному стовпці.

In [16]:
# Отримуємо кількість відсутніх елементів даних по стовпцях
missing_data = data_for_cleaning.isnull().sum()
missing_data[0:4] # дані пропущені у "Peak" та у "All Time Peak" у кількості 11 та 14 відповідно

Rank              0
Peak             11
All Time Peak    14
Actual gross      0
dtype: int64

In [20]:
# Визначаємо загальну кількість відсутніх елементів даних
total_cells = np.product(data_for_cleaning.shape)
total_missing = missing_data.sum()

# Визначаємо відсоток відсутніх елементів даних
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

11.363636363636363


## Третій етап. 
#### Визначення причини відсутності елементів даних. 
##### - Якщо значення відсутні випадково (помилка збору або введення даних), їх можна імплементувати.
##### - Якщо значення відсутнє за визначенням (наприклад, поле не має сенсу для певних об’єктів) — залишаємо NaN.
##### - Можливо нашому датасеті потрібно аналізувати кожен стовпець окремо, щоб визначити природу відсутніх даних.

#### У нашому випадку дані просто відсутні за визначенням, вони не були відсутні випадково. Нашим датасетом не було передбачено відображення місця у топі пісні, яка зайняла низьким рейтинг.


In [5]:
# Виведемо кілька рядків з пропусками для аналізу
data_for_cleaning[data_for_cleaning.isnull().any(axis=1)].head(10) # дані відсутні через не існування відповідних даних 

,Rank,Peak,All Time Peak,Actual gross,Actual gross(in 2022 dollars),Artist,Tour title,Year(s),Shows,Average gross,Ref.
4,5,2[4],NaN,"$345,675,146","$402,844,849",Taylor Swift,Reputation Stadium Tour,2018,53,"$6,522,173",[8]
6,7,2[10],NaN,"$280,000,000","$381,932,682",Celine Dion,Taking Chances World Tour,2008–2009,131,"$2,137,405",[11]
7,7,NaN,NaN,"$257,600,000","$257,600,000",Pink,Summer Carnival †,2023–2024,41,"$6,282,927",[12]
8,9,NaN,NaN,"$256,084,556","$312,258,401",Beyoncé,The Formation World Tour,2016,49,"$5,226,215",[13]
9,10,NaN,NaN,"$250,400,000","$309,141,878",Taylor Swift,The 1989 World Tour,2015,85,"$2,945,882",[14]
10,11,NaN,NaN,"$229,100,000[b]","$283,202,896",Beyoncé,The Mrs. Carter Show World Tour,2013–2014,132,"$1,735,606",[15][16]
11,12,NaN,14[17],"$227,400,000","$295,301,479",Lady Gaga,The Monster Ball Tour *,2009–2011,203,"$1,118,227",[18]
12,13,NaN,NaN,"$204,000,000","$251,856,802",Katy Perry,Prismatic World Tour,2014–2015,151,"$1,350,993",[19]
13,14,1[20],NaN,"$200,000,000","$299,676,265",Cher,Living Proof: The Farewell Tour ‡[21][a],2002–2005,325,"$615,385",[20]
14,15,2[c],NaN,"$194,000,000","$281,617,035",Madonna,Confessions Tour,2006,60,"$3,233,333",[5]


# Четвертий етап. 
#### Обробка пропущених даних. 

In [6]:
# Розмір після видалення рядків з пропусками:
data_drop_rows = data_for_cleaning.dropna()
data_drop_rows.shape

Розмір після видалення рядків з пропусками: (5, 11)


In [13]:
# Звівняння даних повного розміру датасету з пропусками до датасету без пропусків
shape_of_data = data_for_cleaning.shape
shape_of_data

(20, 11)

In [20]:
data_drop_cols = data_for_cleaning.dropna(axis=1)
data_drop_cols.shape

(20, 9)

#### Заповнення пропущених даних
##### Хоча у моєму дата саті доцільніше не заповнювати пропусти, там як дані не втрачені, а просто відсутні. Але для демонстації я проведу декілька маніпуляцій з даними. 
##### Чому недоцільно заповнювати саме мій датасет?!
##### Тому що дані з пропусками присутні саме у місцях рейтенгу за весь час та за наприклад річних "Peak". Можливо пісня і отримала якесь місце у рейтингу, наприклад 200-е, але у даному датасеті не передбачене таке відображення. Інший випадок, коли дана у "Peak" присутні, але відсутні у "All Time Peak". Це відбувається через те, що пісня потрапила у топ за короткий проміжок часу, але не потрапила до загального рейтингу пісень всіх часів.

In [34]:
# Визначення найвищого можливого місця рейтингу датасету
data_for_cleaning["Peak"] = pd.to_numeric(data_for_cleaning["Peak"].astype(str).str.extract(r"(\d+)")[0], errors="coerce")
data_for_cleaning["All Time Peak"] = pd.to_numeric(data_for_cleaning["All Time Peak"].astype(str).str.extract(r"(\d+)")[0], errors="coerce")
data_max_peak = data_for_cleaning[["Peak", "All Time Peak"]].max(axis=1)
highest_peak = data_max_peak.max()
highest_peak

14.0

In [18]:
# Заповнення пропущених даних нулями
data_fill_0 = data_for_cleaning.fillna(0)
data_fill_0

,Rank,Peak,All Time Peak,Actual gross,Actual gross(in 2022 dollars),Artist,Tour title,Year(s),Shows,Average gross,Ref.
0,1,1,2,"$780,000,000","$780,000,000",Taylor Swift,The Eras Tour †,2023–2024,56,"$13,928,571",[1]
1,2,1,7[2],"$579,800,000","$579,800,000",Beyoncé,Renaissance World Tour,2023,56,"$10,353,571",[3]
2,3,1[4],2[5],"$411,000,000","$560,622,615",Madonna,Sticky & Sweet Tour ‡[4][a],2008–2009,85,"$4,835,294",[6]
3,4,2[7],10[7],"$397,300,000","$454,751,555",Pink,Beautiful Trauma World Tour,2018–2019,156,"$2,546,795",[7]
4,5,2[4],0,"$345,675,146","$402,844,849",Taylor Swift,Reputation Stadium Tour,2018,53,"$6,522,173",[8]
5,6,2[4],10[9],"$305,158,363","$388,978,496",Madonna,The MDNA Tour,2012,88,"$3,467,709",[9]
6,7,2[10],0,"$280,000,000","$381,932,682",Celine Dion,Taking Chances World Tour,2008–2009,131,"$2,137,405",[11]
7,7,0,0,"$257,600,000","$257,600,000",Pink,Summer Carnival †,2023–2024,41,"$6,282,927",[12]
8,9,0,0,"$256,084,556","$312,258,401",Beyoncé,The Formation World Tour,2016,49,"$5,226,215",[13]
9,10,0,0,"$250,400,000","$309,141,878",Taylor Swift,The 1989 World Tour,2015,85,"$2,945,882",[14]


In [6]:
# Копія, щоб не зіпсувати оригінал
data_fill_mean = data_for_cleaning.copy()

# Функція для витягування першого числа
def extract_first_number(x):
    if pd.isna(x):
        return np.nan
    match = re.search(r'\d+', str(x))
    return float(match.group()) if match else np.nan

# Обробляємо тільки потрібні стовпці
for col in ["Peak", "All Time Peak"]:
    # 1. Витягуємо числа
    numeric_col = data_fill_mean[col].apply(extract_first_number)
    mean_val = numeric_col.mean()
    numeric_col = numeric_col.fillna(mean_val)
    data_fill_mean[col] = numeric_col
    
data_fill_mean

,Rank,Peak,All Time Peak,Actual gross,Actual gross(in 2022 dollars),Artist,Tour title,Year(s),Shows,Average gross,Ref.
0,1,1.000000,2.0,"$780,000,000","$780,000,000",Taylor Swift,The Eras Tour †,2023–2024,56,"$13,928,571",[1]
1,2,1.000000,7.0,"$579,800,000","$579,800,000",Beyoncé,Renaissance World Tour,2023,56,"$10,353,571",[3]
2,3,1.000000,2.0,"$411,000,000","$560,622,615",Madonna,Sticky & Sweet Tour ‡[4][a],2008–2009,85,"$4,835,294",[6]
3,4,2.000000,10.0,"$397,300,000","$454,751,555",Pink,Beautiful Trauma World Tour,2018–2019,156,"$2,546,795",[7]
4,5,2.000000,7.5,"$345,675,146","$402,844,849",Taylor Swift,Reputation Stadium Tour,2018,53,"$6,522,173",[8]
5,6,2.000000,10.0,"$305,158,363","$388,978,496",Madonna,The MDNA Tour,2012,88,"$3,467,709",[9]
6,7,2.000000,7.5,"$280,000,000","$381,932,682",Celine Dion,Taking Chances World Tour,2008–2009,131,"$2,137,405",[11]
7,7,1.555556,7.5,"$257,600,000","$257,600,000",Pink,Summer Carnival †,2023–2024,41,"$6,282,927",[12]
8,9,1.555556,7.5,"$256,084,556","$312,258,401",Beyoncé,The Formation World Tour,2016,49,"$5,226,215",[13]
9,10,1.555556,7.5,"$250,400,000","$309,141,878",Taylor Swift,The 1989 World Tour,2015,85,"$2,945,882",[14]


In [23]:
# Створення стовпцю з чистими даними "Peak"
data_for_cleaning["Peak Clean"] = data_for_cleaning["Peak"].str.extract(r"(\d+)")
data_for_cleaning

,Rank,Peak,All Time Peak,Actual gross,Actual gross(in 2022 dollars),Artist,Tour title,Year(s),Shows,Average gross,Ref.,Peak Clean
0,1,1,2,"$780,000,000","$780,000,000",Taylor Swift,The Eras Tour †,2023–2024,56,"$13,928,571",[1],1
1,2,1,7[2],"$579,800,000","$579,800,000",Beyoncé,Renaissance World Tour,2023,56,"$10,353,571",[3],1
2,3,1[4],2[5],"$411,000,000","$560,622,615",Madonna,Sticky & Sweet Tour ‡[4][a],2008–2009,85,"$4,835,294",[6],1
3,4,2[7],10[7],"$397,300,000","$454,751,555",Pink,Beautiful Trauma World Tour,2018–2019,156,"$2,546,795",[7],2
4,5,2[4],NaN,"$345,675,146","$402,844,849",Taylor Swift,Reputation Stadium Tour,2018,53,"$6,522,173",[8],2
5,6,2[4],10[9],"$305,158,363","$388,978,496",Madonna,The MDNA Tour,2012,88,"$3,467,709",[9],2
6,7,2[10],NaN,"$280,000,000","$381,932,682",Celine Dion,Taking Chances World Tour,2008–2009,131,"$2,137,405",[11],2
7,7,NaN,NaN,"$257,600,000","$257,600,000",Pink,Summer Carnival †,2023–2024,41,"$6,282,927",[12],NaN
8,9,NaN,NaN,"$256,084,556","$312,258,401",Beyoncé,The Formation World Tour,2016,49,"$5,226,215",[13],NaN
9,10,NaN,NaN,"$250,400,000","$309,141,878",Taylor Swift,The 1989 World Tour,2015,85,"$2,945,882",[14],NaN


In [25]:
# Визначення середнього значення та заповнення нового стовпцю "Peak Filled". 
data_for_cleaning["Peak Clean"] = pd.to_numeric(data_for_cleaning["Peak Clean"], errors="coerce")
mean_val = data_for_cleaning["Peak Clean"].mean()
data_for_cleaning["Peak Filled"] = data_for_cleaning["Peak Clean"].fillna(mean_val) # стовпець"Peak Filled" - готовий результат заповнених даних середніми значення
data_for_cleaning 

,Rank,Peak,All Time Peak,Actual gross,Actual gross(in 2022 dollars),Artist,Tour title,Year(s),Shows,Average gross,Ref.,Peak Clean,Peak Filled
0,1,1,2,"$780,000,000","$780,000,000",Taylor Swift,The Eras Tour †,2023–2024,56,"$13,928,571",[1],1.0,1.000000
1,2,1,7[2],"$579,800,000","$579,800,000",Beyoncé,Renaissance World Tour,2023,56,"$10,353,571",[3],1.0,1.000000
2,3,1[4],2[5],"$411,000,000","$560,622,615",Madonna,Sticky & Sweet Tour ‡[4][a],2008–2009,85,"$4,835,294",[6],1.0,1.000000
3,4,2[7],10[7],"$397,300,000","$454,751,555",Pink,Beautiful Trauma World Tour,2018–2019,156,"$2,546,795",[7],2.0,2.000000
4,5,2[4],NaN,"$345,675,146","$402,844,849",Taylor Swift,Reputation Stadium Tour,2018,53,"$6,522,173",[8],2.0,2.000000
5,6,2[4],10[9],"$305,158,363","$388,978,496",Madonna,The MDNA Tour,2012,88,"$3,467,709",[9],2.0,2.000000
6,7,2[10],NaN,"$280,000,000","$381,932,682",Celine Dion,Taking Chances World Tour,2008–2009,131,"$2,137,405",[11],2.0,2.000000
7,7,NaN,NaN,"$257,600,000","$257,600,000",Pink,Summer Carnival †,2023–2024,41,"$6,282,927",[12],NaN,1.555556
8,9,NaN,NaN,"$256,084,556","$312,258,401",Beyoncé,The Formation World Tour,2016,49,"$5,226,215",[13],NaN,1.555556
9,10,NaN,NaN,"$250,400,000","$309,141,878",Taylor Swift,The 1989 World Tour,2015,85,"$2,945,882",[14],NaN,1.555556


#### Заповнення даних через спостереження, що є найбільш доцільним у моєму випадку.
##### Я помітив, що топ у розумінні мого датасету це до 14 місця включно (на жаль точність залежить від велечини датасету, так як точність залежить від велечини вибірки і точно сказати про рейтинг не можливо), тобто всі інші пісні, де місце у рейтингу не вказане займають 14+ місце, це може бути як 11 так і 45. Тому правильно було б так і вказати, що ця пісня зайняла якесь умовне "14+" місце

In [37]:
data_for_cleaning = pd.read_csv("my_file.csv") # відновлення початкових значень
data_for_cleaning["Peak"] = data_for_cleaning["Peak"].fillna("14+") # заповнення пропусків у "Peak" значенням з спостереження
data_for_cleaning["All Time Peak"] = data_for_cleaning["All Time Peak"].fillna("14+") # заповнення пропусків у "All Time Peak" значенням з спостереження
data_for_cleaning

,Rank,Peak,All Time Peak,Actual gross,Actual gross(in 2022 dollars),Artist,Tour title,Year(s),Shows,Average gross,Ref.
0,1,1,2,"$780,000,000","$780,000,000",Taylor Swift,The Eras Tour †,2023–2024,56,"$13,928,571",[1]
1,2,1,7[2],"$579,800,000","$579,800,000",Beyoncé,Renaissance World Tour,2023,56,"$10,353,571",[3]
2,3,1[4],2[5],"$411,000,000","$560,622,615",Madonna,Sticky & Sweet Tour ‡[4][a],2008–2009,85,"$4,835,294",[6]
3,4,2[7],10[7],"$397,300,000","$454,751,555",Pink,Beautiful Trauma World Tour,2018–2019,156,"$2,546,795",[7]
4,5,2[4],14+,"$345,675,146","$402,844,849",Taylor Swift,Reputation Stadium Tour,2018,53,"$6,522,173",[8]
5,6,2[4],10[9],"$305,158,363","$388,978,496",Madonna,The MDNA Tour,2012,88,"$3,467,709",[9]
6,7,2[10],14+,"$280,000,000","$381,932,682",Celine Dion,Taking Chances World Tour,2008–2009,131,"$2,137,405",[11]
7,7,14+,14+,"$257,600,000","$257,600,000",Pink,Summer Carnival †,2023–2024,41,"$6,282,927",[12]
8,9,14+,14+,"$256,084,556","$312,258,401",Beyoncé,The Formation World Tour,2016,49,"$5,226,215",[13]
9,10,14+,14+,"$250,400,000","$309,141,878",Taylor Swift,The 1989 World Tour,2015,85,"$2,945,882",[14]


## Висновок

У ході виконання лабораторної роботи було опрацьовано набір “брудних” даних, що містив пропуски, некоректні значення та неоднорідні формати записів. Для їх обробки було застосовано інструменти бібліотеки Pandas, зокрема методи для перевірки наявності пропусків, заміни відсутніх значень, приведення даних до правильного типу, а також видалення або корекції помилкових рядків.

Практична реалізація показала, що етап Data Cleaning є критично важливим у процесі аналізу даних: саме якість підготовленого датасету визначає подальшу точність і коректність результатів. У результаті роботи вдалося перетворити “сирі” дані на більш структуровані та готові до подальшої аналітики.

Таким чином, було досягнуто поставленої мети: набуті навички роботи з пропущеними й некоректними значеннями, а також засвоєно основні прийоми попередньої обробки даних у Python.